In [0]:
#!pip install pydrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
import linecache
import os
import sys
from tqdm import tqdm
import random

from collections import Counter
from six.moves.urllib.request import urlretrieve

random.seed(42)

tinyimagenet_base_url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"

def reporthook(t):
  """https://github.com/tqdm/tqdm"""
  last_b = [0]

  def inner(b=1, bsize=1, tsize=None):
    """
    b: int, optional
        Number of blocks just transferred [default: 1].
    bsize: int, optional
        Size of each block (in tqdm units) [default: 1].
    tsize: int, optional
        Total size (in tqdm units). If [default: None] remains unchanged.
    """
    if tsize is not None:
        t.total = tsize
    t.update((b - last_b[0]) * bsize)
    last_b[0] = b
  return inner

def maybe_download(url, filename, prefix, num_bytes=None):
    """Takes an URL, a filename, and the expected bytes, download
    the contents and returns the filename
    num_bytes=None disables the file size check."""
    local_filename = None
    if not os.path.exists(os.path.join(prefix, filename)):
        try:
            print("Downloading file {}...".format(url + filename))
            with tqdm(unit='B', unit_scale=True, miniters=1, desc=filename) as t:
                local_filename, _ = urlretrieve(url + filename, os.path.join(prefix,filename), reporthook=reporthook(t))
        except AttributeError as e:
            print("An error occurred when downloading the file! Please get the dataset using a browser.")
            raise e
    # We have a downloaded file
    # Check the stats and make sure they are ok
    file_stats = os.stat(os.path.join(prefix,filename))
    if num_bytes is None or file_stats.st_size == num_bytes:
        print("File {} successfully loaded".format(filename))
    else:
        raise Exception("Unexpected dataset size. Please get the dataset using a browser.")

    return local_filename

In [0]:
import zipfile
import shutil


if __name__ == '__main__':
    tinyimagenet_base_url = "http://cs231n.stanford.edu/"
    tinyimagenet_filename = "tiny-imagenet-200.zip"
    unzipped_filename = "tiny-imagenet-200"

    prefix = os.path.join("download", "dwr")
    data_prefix = os.path.join("data")

    print("Storing datasets in {}".format(prefix))

    if not os.path.exists(prefix):
        os.makedirs(prefix)
    if not os.path.exists(data_prefix):
        os.makedirs(data_prefix)
    
    glove_zip = maybe_download(tinyimagenet_base_url, tinyimagenet_filename, prefix, None)

    print("Extracting")
    glove_zip_ref = zipfile.ZipFile(os.path.join(prefix, tinyimagenet_filename), 'r')

    glove_zip_ref.extractall(prefix)
    glove_zip_ref.close()
    
    print("Moving datasets to {}".format(data_prefix))
    shutil.move(os.path.join(prefix, unzipped_filename), os.path.join(data_prefix, unzipped_filename))

Storing datasets in download/dwr
File tiny-imagenet-200.zip successfully loaded
Extracting
Moving datasets to data


In [0]:
# ###################################################### This is the Block WIth 32x32 and no AUgmentation Images  ####################################
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

BATCH_SIZE=500

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    #zoom_range = 0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #shear_range=0.2,
    #fill_mode='nearest'
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./data/tiny-imagenet-200/train/', target_size=(32,32),  
                                                    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
val_data = pd.read_csv(r'./data/tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
VAL_BATCH_SIZE=500
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory=r'./data/tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                     class_mode='categorical', batch_size=VAL_BATCH_SIZE, shuffle=True)

Using TensorFlow backend.


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
from keras import backend as tf

###################################################### This is the Block WIth Custom Architecure, No ResNet ,But Rreplacement of Dense Layer withGAP for 200 CLasses    ####################################
# backend
import tensorflow as tf
from keras import backend as k
from keras.regularizers import l2
from keras import backend as K

from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=4)

# This can be done to support Multiple Shape Types as Input
input = Input(shape=(None,None,3,))

#def space_to_depth_x2(x):
#    return tf.space_to_depth(x, block_size=8)
#input = Input(shape=(64,64,3))

from keras.layers import SeparableConv2D,DepthwiseConv2D,Convolution2D,GlobalAveragePooling2D
#Block1
layer1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2_block_32', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1_block_1')(layer1)
layer1 = ReLU()(layer1)

# Layer 2
layer2 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_2_block_1', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_1_block_layer2')(layer2)
layer2 = ReLU()(layer2)



layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4_block_1', use_bias=False)(layer2)
layer4 = BatchNormalization(name='norm_1_4_block_1')(layer4)
layer4 = ReLU()(layer4)

layer5 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4_16', use_bias=False)(layer4)
layer5 = BatchNormalization(name='norm_4_16')(layer5)
layer5 = ReLU()(layer5)

layer5_Max = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection = layer5_Max


# Layer 3
layer6 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3_block_2', use_bias=False)(layer5_Max)
layer6 = BatchNormalization(name='norm_1_3_block_2')(layer6)
layer6 = ReLU()(layer6)

#layer4

layer6_1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_3_block_2_1', use_bias=False)(layer6)
layer6_1 = BatchNormalization(name='norm_1_3_block_2_1')(layer6_1)
layer6_1 = ReLU()(layer6_1)

layer6_2 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_3_block_2_2', use_bias=False)(layer6_1)
layer6_2 = BatchNormalization(name='norm_1_3_block_2_2')(layer6_2)
layer6_2 = ReLU()(layer6_2)

#layer4

layer14_64_21 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7_block_3_1_layer14_62', use_bias=False)(layer6_2)
layer14_64_21 = BatchNormalization(name='conv_7_block_3_1_layer14_611')(layer14_64_21)
layer14_64_21 = ReLU()(layer14_64_21)


layer14_Max_64 = MaxPooling2D(pool_size=(2, 2))(layer14_64_21)



layer14_64_2 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_7_block__1', use_bias=False)(layer14_Max_64)
layer14_64_2 = BatchNormalization(name='norm_14_layr4_64')(layer14_64_2)
layer14_64_2 = ReLU()(layer14_64_2)


layer14_2 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_7_block__1a', use_bias=False)(layer14_64_2)
layer14_2 = BatchNormalization(name='norm_14_layr4_64a')(layer14_2)
layer14_2 = ReLU()(layer14_2)


layer14_64_256 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='layer14_64_256A', use_bias=False)(layer14_2)
layer14_64_256 = BatchNormalization(name='layer14_64_256')(layer14_64_256)
layer14_64_256 = ReLU()(layer14_64_256)

layer14_Max_256 = MaxPooling2D(pool_size=(2, 2))(layer14_64_256)


# Layer 15
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='skipLayer', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='skip_connection')(skip_connection)
skip_connection = ReLU()(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

layer15 = concatenate([skip_connection, layer14_Max_256])



layer19_1_Max_O_2_K = MaxPooling2D(pool_size=(2,2))(layer15)




layer20 = Convolution2D(200,(1,1),strides=(1,1),padding='same', name='conv_1EP', use_bias=False)(layer19_1_Max_O_2_K)
# This is the replacement of Dense Layer
layer21=GlobalAveragePooling2D(data_format='channels_last')(layer20)

output= Activation('softmax')(layer21)


model = Model(inputs=[input], outputs=[output])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv_2_block_32 (Conv2D)        (None, None, None, 6 1728        input_1[0][0]                    
__________________________________________________________________________________________________
norm_1_block_1 (BatchNormalizat (None, None, None, 6 256         conv_2_block_32[0][0]            
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, None, None, 6 0           norm_1_block_1[0][0]             
_____________________________________

In [0]:
#Model Fit Generator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint
reduce = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

filepath = "/content/gdrive/My Drive/model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)

callbacks=[reduce,checkpoint]
NUM_EPOCHS=10

# make sure to update the steps_per_epoch & validation_steps properly
# in case you are using custom generator with aug for validation data as well, then replace VAL_IMAGES with validation_batches.sample
# NOTE: do not forget to use // for dividing, as it returns an int. just / returns float
# NOTE: int() in caculating steps_per_epoch & validation_steps is not required as dividing by // returns int
history = model.fit_generator(train_generator,
                    steps_per_epoch=int(train_generator.n//BATCH_SIZE), 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_steps=int(validation_generator.n//BATCH_SIZE), 
                    validation_data=validation_generator,callbacks=callbacks)


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
200/200 [==============================] - 286s 1s/step - loss: 4.7637 - acc: 0.0643 - val_loss: 5.8891 - val_acc: 0.0238

Epoch 00001: loss improved from inf to 4.76370, saving model to /content/gdrive/My Drive/model.h5
Epoch 2/10
200/200 [==============================] - 255s 1s/step - loss: 4.0063 - acc: 0.1478 - val_loss: 5.0056 - val_acc: 0.0586

Epoch 00002: loss improved from 4.76370 to 4.00629, saving model to /content/gdrive/My Drive/model.h5
Epoch 3/10
200/200 [==============================] - 255s 1s/step - loss: 3.6702 - acc: 0.1986 - val_loss: 4.5975 - val_acc: 0.0764

Epoch 00003: loss improved from 4.00629 to 3.67017, saving model to /content/gdrive/My Drive/model.h5
Epoch 4/10
200/200 [==============================] - 255s 1s/step - loss: 3.4066 - acc: 0.2412 - val_loss: 4.2860 - val_acc: 0.1161

Epoch 00004: loss improved from 3.67017 to 3.40663, saving model to /content/gdrive/My Drive/model.h5
Epoch 5/10
2

In [0]:
# This is the Block which will train for 16x16 images and validation of 64x64 with no Augmnetation
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

BATCH_SIZE=400

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    #zoom_range = 0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #shear_range=0.2,
    #fill_mode='nearest'
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./data/tiny-imagenet-200/train/', target_size=(16,16),  
                                                    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
val_data = pd.read_csv(r'./data/tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory=r'./data/tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                     class_mode='categorical', batch_size=BATCH_SIZE, shuffle=True)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
# It will run for the above 16x16 block with 10 epochs and no augmnetation
from keras.regularizers import l2

from keras.backend import tf
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
# use the generators as usual in fit_generator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint

new_model = load_model("/content/gdrive/My Drive/model.h5", custom_objects={'tf': tf})
NUM_EPOCHS=10
filepath="model_16_10From_model.h5"
# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
clr_triangular = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
callbacks = [checkpoint,clr_triangular]
new_model.fit_generator(train_generator,
                    steps_per_epoch=int(train_generator.n//BATCH_SIZE), 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_steps=int(validation_generator.n//BATCH_SIZE), 
  
                    validation_data=validation_generator,callbacks=callbacks)


Epoch 1/10
250/250 [==============================] - 153s 612ms/step - loss: 3.3976 - acc: 0.2503 - val_loss: 5.9448 - val_acc: 0.0525

Epoch 00001: loss improved from inf to 3.39761, saving model to model_16_10From_model.h5
Epoch 2/10
250/250 [==============================] - 133s 531ms/step - loss: 3.0497 - acc: 0.3102 - val_loss: 5.8402 - val_acc: 0.0468

Epoch 00002: loss improved from 3.39761 to 3.04969, saving model to model_16_10From_model.h5
Epoch 3/10
250/250 [==============================] - 132s 527ms/step - loss: 2.8092 - acc: 0.3509 - val_loss: 6.3266 - val_acc: 0.0394

Epoch 00003: loss improved from 3.04969 to 2.80916, saving model to model_16_10From_model.h5
Epoch 4/10
250/250 [==============================] - 132s 528ms/step - loss: 2.5405 - acc: 0.4004 - val_loss: 6.0858 - val_acc: 0.0469

Epoch 00004: loss improved from 2.80916 to 2.54046, saving model to model_16_10From_model.h5
Epoch 5/10
250/250 [==============================] - 132s 527ms/step - loss: 2.3222

In [0]:
###################################################### This is the  Block, WIth 16x16 Images and zoom_range of 0.2 with BATCH SIZE of 400 ####################################
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

BATCH_SIZE=400

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range = 0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #shear_range=0.2,
    #fill_mode='nearest'
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./data/tiny-imagenet-200/train/', target_size=(16,16),  
                                                    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
val_data = pd.read_csv(r'./data/tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory=r'./data/tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                     class_mode='categorical', batch_size=BATCH_SIZE, shuffle=True)
from keras.regularizers import l2

from keras.backend import tf
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
# use the generators as usual in fit_generator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint

new_model = load_model("model_16_10From_model.h5", custom_objects={'tf': tf})
NUM_EPOCHS=10
filepath="/content/gdrive/My Drive/model_16_10From_model_zoom.h5"
# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
clr_triangular = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
callbacks = [checkpoint,clr_triangular]
new_model.fit_generator(train_generator,
                    steps_per_epoch=int(train_generator.n//BATCH_SIZE), 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_steps=int(validation_generator.n//BATCH_SIZE), 
  
                    validation_data=validation_generator,callbacks=callbacks)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/10
250/250 [==============================] - 168s 671ms/step - loss: 1.7960 - acc: 0.5326 - val_loss: 6.9510 - val_acc: 0.0436

Epoch 00001: loss improved from inf to 1.79596, saving model to /content/gdrive/My Drive/model_16_10From_model_zoom.h5
Epoch 2/10
250/250 [==============================] - 159s 638ms/step - loss: 1.5934 - acc: 0.5799 - val_loss: 6.8476 - val_acc: 0.0484

Epoch 00002: loss improved from 1.79596 to 1.59341, saving model to /content/gdrive/My Drive/model_16_10From_model_zoom.h5
Epoch 3/10
250/250 [==============================] - 159s 634ms/step - loss: 1.4478 - acc: 0.6169 - val_loss: 7.1089 - val_acc: 0.0441

Epoch 00003: loss improved from 1.59341 to 1.44777, saving model to /content/gdrive/My Drive/model_16_10From_model_zoom.h5
Epoch 4/10
250/250 [==============================] - 158s 630ms/step - loss: 1.3356 - acc: 0.6432 - val_loss: 6.9263 - val_acc: 0.04

In [0]:
###################################################### This is the  Block, WIth 56x56 Images and BATCH SIZE=250 ####################################
                                                    #Batch SIz reduced Otherwise gets OOME with Increased Image SIze.
# Use Augmentaion parameters as required.
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

BATCH_SIZE=250

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    #zoom_range = 0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #shear_range=0.2,
    #fill_mode='nearest'
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./data/tiny-imagenet-200/train/', target_size=(56,56),  
                                                    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
val_data = pd.read_csv(r'./data/tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory=r'./data/tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                     class_mode='categorical', batch_size=BATCH_SIZE, shuffle=True)
from keras.regularizers import l2

from keras.backend import tf
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
# use the generators as usual in fit_generator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint

new_model = load_model("/content/gdrive/My Drive/model_16_10From_model_zoom.h5", custom_objects={'tf': tf})
NUM_EPOCHS=10
filepath="/content/gdrive/My Drive/model_56.h5"
# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
clr_triangular = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
callbacks = [checkpoint,clr_triangular]
new_model.fit_generator(train_generator,
                    steps_per_epoch=int(train_generator.n//BATCH_SIZE), 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_steps=int(validation_generator.n//BATCH_SIZE), 
  
                    validation_data=validation_generator,callbacks=callbacks)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/10
400/400 [==============================] - 756s 2s/step - loss: 2.9541 - acc: 0.3111 - val_loss: 3.0415 - val_acc: 0.2968

Epoch 00001: loss improved from inf to 2.95408, saving model to /content/gdrive/My Drive/model_56.h5
Epoch 2/10
400/400 [==============================] - 732s 2s/step - loss: 2.5067 - acc: 0.3975 - val_loss: 2.8605 - val_acc: 0.3285

Epoch 00002: loss improved from 2.95408 to 2.50674, saving model to /content/gdrive/My Drive/model_56.h5
Epoch 3/10
400/400 [==============================] - 731s 2s/step - loss: 2.3094 - acc: 0.4388 - val_loss: 2.7357 - val_acc: 0.3549

Epoch 00003: loss improved from 2.50674 to 2.30941, saving model to /content/gdrive/My Drive/model_56.h5
Epoch 4/10
400/400 [==============================] - 733s 2s/step - loss: 2.1676 - acc: 0.4705 - val_loss: 2.6556 - val_acc: 0.3726

Epoch 00004: loss improved from 2.30941 to 2.16765, saving mod

In [0]:
###################################################### This is the  Block, WIth 64x64 Images Only and Validat_accuract of 49 with accu: 90(OVERFITTING) and BATCH SIZE 200 #######
# Use Augmentaion parameters as required.
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

BATCH_SIZE=200

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    #zoom_range = 0.2,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #shear_range=0.2,
    #fill_mode='nearest'
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./data/tiny-imagenet-200/train/', target_size=(64,64),  
                                                    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)
val_data = pd.read_csv(r'./data/tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory=r'./data/tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                     class_mode='categorical', batch_size=150, shuffle=True)
from keras.regularizers import l2

from keras.backend import tf
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
# use the generators as usual in fit_generator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint

new_model = load_model("/content/gdrive/My Drive/model_56.h5", custom_objects={'tf': tf})
print(new_model)
NUM_EPOCHS=20
filepath="/content/gdrive/My Drive/model_64.h5"
# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
clr_triangular = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=12, min_lr=0.5e-6)
callbacks = [checkpoint,clr_triangular]
new_model.fit_generator(train_generator,
                    steps_per_epoch=int(train_generator.n//BATCH_SIZE), 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_steps=int(validation_generator.n//BATCH_SIZE), 
  
                    validation_data=validation_generator,callbacks=callbacks)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/20
500/500 [==============================] - 939s 2s/step - loss: 1.9322 - acc: 0.5250 - val_loss: 2.3422 - val_acc: 0.4431

Epoch 00001: val_acc improved from -inf to 0.44307, saving model to /content/gdrive/My Drive/model_64.h5
Epoch 2/20
500/500 [==============================] - 939s 2s/step - loss: 1.7973 - acc: 0.5541 - val_loss: 2.3329 - val_acc: 0.4400

Epoch 00002: val_acc did not improve from 0.44307
Epoch 3/20
500/500 [==============================] - 934s 2s/step - loss: 1.6977 - acc: 0.5764 - val_loss: 2.2862 - val_acc: 0.4499

Epoch 00003: val_acc improved from 0.44307 to 0.44993, saving model to /content/gdrive/My Drive/model_64.h5
Epoch 4/20
500/500 [==============================] - 934s 2s/step - loss: 1.6086 - acc: 0.5991 - val_loss: 2.2592 - val_acc: 0.4625

Epoch 00004: val_acc improved from 0.44993 to 0.46253, saving model to /content/gdrive/My Drive/model_64.h5
Ep

In [0]:
#CYCLIC LEARNIN RATE
from keras.callbacks import Callback
from keras import backend as K
import numpy as np


class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency.
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.

    The amplitude of the cycle can be scaled on a per-iteration or
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each
        cycle iteration.
    For more detail, please see paper.

    # Example for CIFAR-10 w/ batch size 100:
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```

    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```

    # References

      - [Cyclical Learning Rates for Training Neural Networks](
      https://arxiv.org/abs/1506.01186)
    """

    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.006,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2.**(x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [0]:
###################################################### This is the final Block, WIth 64x64 Images, CYCLIC LEARNING RATE and IMAGE AUGMENTATION ####################################
# Use Augmentaion parameters as required.
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2

from keras.backend import tf
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import ReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
# use the generators as usual in fit_generator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint

from imgaug import augmenters as iaa
import imgaug as ia
from sklearn.utils import class_weight




sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.CropAndPad(
            #px=((0, 30), (0, 10), (0, 30), (0, 10)),
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 256)
            ),
        
       
        iaa.Flipud(0.2), # vertically flip 20% of all images

        # crop some of the images by 0-10% of their height/width
        sometimes(iaa.Crop(percent=(0, 0.5))),
                      # Sharpen each image, overlay the result with the original
                # image using an alpha between 0 (no sharpening) and 1
                # (full sharpening effect).
        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.5, 1.5)),

                # Same as sharpen, but for an embossing effect.
        iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
        iaa.CoarseDropout((0.0,0.03),size_percent=(0.01,0.15)),
        iaa.Multiply((0.5, 1.5), per_channel=0.5)])


#BATCH SIZE
BATCH_SIZE=200

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    #zoom_range = 0.2,
    preprocessing_function=seq.augment_image
    #fill_mode='nearest'
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #shear_range=0.2,
    
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./data/tiny-imagenet-200/train/', target_size=(64,64),  
                                                    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True,
                                                    color_mode='rgb',interpolation='bilinear')


print(train_generator)




val_data = pd.read_csv(r'./data/tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory=r'./data/tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                     class_mode='categorical', batch_size=150, shuffle=True)

new_model_64 = load_model("/content/gdrive/My Drive/model_64.h5", custom_objects={'tf': tf})
print(new_model_64)
NUM_EPOCHS=50
filepath="/content/gdrive/My Drive/model_64_learn.h5"
# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#CYclic Learning Rate and mode is "Triangular2" HERE STEP_SIZE is increased , THIS PLAYS MAJOR ROLE IN INCREASING THE ACCURACY.
clr = CyclicLR(base_lr=0.00001, max_lr=0.008,step_size=5000, mode='triangular2')

# CallBack For CHeck POint AND Cyclic Learning Rate
callbacks = [checkpoint,clr]

#Load the Fit_generator with previous Trained model that learnt only on 64x64 images.
new_model_64.fit_generator(train_generator,
                    steps_per_epoch=int(train_generator.n//BATCH_SIZE), 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_steps=int(validation_generator.n//150), 
  
                    validation_data=validation_generator,callbacks=callbacks)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/50
500/500 [==============================] - 876s 2s/step - loss: 4.4840 - acc: 0.1202 - val_loss: 3.9100 - val_acc: 0.1589

Epoch 00001: val_acc improved from -inf to 0.15889, saving model to /content/gdrive/My Drive/model_64_learn.h5
Epoch 2/50
500/500 [==============================] - 871s 2s/step - loss: 4.0068 - acc: 0.1604 - val_loss: 3.4676 - val_acc: 0.2189

Epoch 00002: val_acc improved from 0.15889 to 0.21888, saving model to /content/gdrive/My Drive/model_64_learn.h5
Epoch 3/50
500/500 [==============================] - 871s 2s/step - loss: 3.8108 - acc: 0.1806 - val_loss: 3.8406 - val_acc: 0.1698

Epoch 00003: val_acc did not improve from 0.21888
Epoch 4/50
500/500 [==============================] - 870s 2s/step - loss: 3.7034 - acc: 0.1974 - val_loss: 3.5462 - val_acc: 0.2159

Epoch 00004: val_acc did not improve from 0.21888
Epoch 5/50
500/500 [============================